# Deadlines

In [1]:
from configs.config import get_path
import src.utils.converter as convert
import src.utils.presenter as show
import src.utils.checker as check
import pandas as pd

import src.utils.gen_interarrival as gen_interarrival
import math

In [2]:
basic_data_path = get_path("data", "basic")

df_instance = pd.read_csv(basic_data_path / "00_instance.csv")
df_instance

,Job,Operation,Machine,Processing Time
0,job 0,0,M0,29
1,job 0,1,M1,78
2,job 0,2,M2,9
3,job 0,3,M3,36
4,job 0,4,M4,49
...,...,...,...,...
95,job 9,5,M9,76
96,job 9,6,M5,47
97,job 9,7,M3,52
98,job 9,8,M4,90


In [3]:
# Step 1: Gesamtdauer und Wahrscheinlichkeit
job_dauer = df_instance.groupby("Job")["Processing Time"].sum()
job_wahrscheinlichkeit = job_dauer / job_dauer.sum()
job_wahrscheinlichkeit 


Job
job 0    0.077315
job 1    0.099824
job 2    0.111176
job 3    0.128205
job 4    0.076923
job 5    0.097084
job 6    0.081425
job 7    0.105500
job 8    0.116853
job 9    0.105696
Name: Processing Time, dtype: float64

In [4]:
# Step 2: Ankunftsrate
mean_interarrival_time = gen_interarrival.calculate_mean_interarrival_time(df_instance)
lambda_gesamt = 1 / mean_interarrival_time
lambda_j = job_wahrscheinlichkeit * lambda_gesamt

print(f"mittlere Zwischenankunftszeit: {mean_interarrival_time}")
print(f"Ankunftsrate: {lambda_gesamt}")

mittlere Zwischenankunftszeit: 70.11
Ankunftsrate: 0.01426330052774212


In [5]:
# Step 3: Zusammenführen
df_jobs = pd.DataFrame({
    "Job": job_dauer.index,
    "Wahrscheinlichkeit P": job_wahrscheinlichkeit.values,
    "Ankunftsrate λ": lambda_j.values
})
df_merged = df_instance.merge(df_jobs, on="Job")
df_merged

,Job,Operation,Machine,Processing Time,Wahrscheinlichkeit P,Ankunftsrate λ
0,job 0,0,M0,29,0.077315,0.001103
1,job 0,1,M1,78,0.077315,0.001103
2,job 0,2,M2,9,0.077315,0.001103
3,job 0,3,M3,36,0.077315,0.001103
4,job 0,4,M4,49,0.077315,0.001103
...,...,...,...,...,...,...
95,job 9,5,M9,76,0.105696,0.001508
96,job 9,6,M5,47,0.105696,0.001508
97,job 9,7,M3,52,0.105696,0.001508
98,job 9,8,M4,90,0.105696,0.001508


In [6]:
# Step 4: Beiträge zur Kontrollrechnung für jede Maschine
df_merged["Pj * d_jm"] = df_merged["Wahrscheinlichkeit P"] * df_merged["Processing Time"] # Zähler
df_merged["λj * d_jm"] = df_merged["Ankunftsrate λ"] * df_merged["Processing Time"]       # Nenner

# Step 5: Summieren nach Maschine
df_control = df_merged.groupby("Machine")[["Pj * d_jm", "λj * d_jm"]].sum()
df_control["Kontrollwert"] = df_control["Pj * d_jm"] / df_control["λj * d_jm"]

# Engpassmaschine
engpass = df_control["λj * d_jm"].idxmax()
kontrollwert_engpass = df_control.loc[engpass, "Kontrollwert"]

# Ausgabe
print(f"\n🛠️ Engpassmaschine: {engpass}")
print(f"🎯 Kontrollwert (soll ≈ mean_interarrival_time): {kontrollwert_engpass:.4f}")
df_control.sort_values("Kontrollwert", ascending=False)


🛠️ Engpassmaschine: M3
🎯 Kontrollwert (soll ≈ mean_interarrival_time): 70.1100


,Pj * d_jm,λj * d_jm,Kontrollwert
Machine,,,
M0,52.452143,0.748141,70.11
M2,60.027403,0.856189,70.11
M8,54.007438,0.770324,70.11
M3,65.472500,0.933854,70.11
M5,42.251713,0.602649,70.11
M6,48.124878,0.686420,70.11
M9,40.061460,0.571409,70.11
M7,51.341750,0.732303,70.11
M1,56.859268,0.811001,70.11


# Excel Beispiel

In [7]:
# Nach Reset: Daten erneut definieren
routing_data = {
    "Routing_1": [["M1", 2.0], ["M2", 5.0], ["M3", 4.0]],
    "Routing_2": [["M2", 2.0], ["M3", 3.0], ["M1", 5.0]],
    "Routing_3": [["M3", 4.0], ["M1", 2.0], ["M2", 3.0]]
}

# Umwandlung in DataFrame
records = []
for job, operations in routing_data.items():
    for machine, proc_time in operations:
        records.append({
            "Job": job,
            "Machine": machine,
            "Processing Time": proc_time
        })

df_instance = pd.DataFrame(records)
df_instance 

,Job,Machine,Processing Time
0,Routing_1,M1,2.0
1,Routing_1,M2,5.0
2,Routing_1,M3,4.0
3,Routing_2,M2,2.0
4,Routing_2,M3,3.0
5,Routing_2,M1,5.0
6,Routing_3,M3,4.0
7,Routing_3,M1,2.0
8,Routing_3,M2,3.0


In [8]:
gen_interarrival.calculate_mean_interarrival_time(df_instance)

4.07

In [9]:
# Step 1: Gesamtdauer und Wahrscheinlichkeit
job_dauer = df_instance.groupby("Job")["Processing Time"].sum()
job_wahrscheinlichkeit = job_dauer / job_dauer.sum()

# Step 2: Ankunftsrate
mean_interarrival_time = 4.31372549
lambda_gesamt = 1 / mean_interarrival_time
lambda_j = job_wahrscheinlichkeit * lambda_gesamt

# Step 3: Zusammenführen
df_jobs = pd.DataFrame({
    "Job": job_dauer.index,
    "Wahrscheinlichkeit P": job_wahrscheinlichkeit.values,
    "Ankunftsrate λ": lambda_j.values
})
df_merged = df_instance.merge(df_jobs, on="Job")

# Step 4: Beiträge zur Kontrollrechnung für jede Maschine
df_merged["Pj * d_jm"] = df_merged["Wahrscheinlichkeit P"] * df_merged["Processing Time"] # Zähler
df_merged["λj * d_jm"] = df_merged["Ankunftsrate λ"] * df_merged["Processing Time"]       # Nenner

# Step 5: Summieren nach Maschine
df_control = df_merged.groupby("Machine")[["Pj * d_jm", "λj * d_jm"]].sum()
df_control["Kontrollwert"] = df_control["Pj * d_jm"] / df_control["λj * d_jm"]

# Engpassmaschine
engpass = df_control["λj * d_jm"].idxmax()
kontrollwert_engpass = df_control.loc[engpass, "Kontrollwert"]

# Ausgabe
print(f"\nEngpassmaschine: {engpass}")
print(f"Kontrollwert (soll ≈ mean_interarrival_time): {kontrollwert_engpass:.4f}")
df_control.sort_values("Kontrollwert", ascending=False)


Engpassmaschine: M3
Kontrollwert (soll ≈ mean_interarrival_time): 4.3137


,Pj * d_jm,λj * d_jm,Kontrollwert
Machine,,,
M1,3.000000,0.695455,4.313725
M2,3.400000,0.788182,4.313725
M3,3.666667,0.850000,4.313725


In [10]:
df_merged

,Job,Machine,Processing Time,Wahrscheinlichkeit P,Ankunftsrate λ,Pj * d_jm,λj * d_jm
0,Routing_1,M1,2.0,0.366667,0.085000,0.733333,0.170000
1,Routing_1,M2,5.0,0.366667,0.085000,1.833333,0.425000
2,Routing_1,M3,4.0,0.366667,0.085000,1.466667,0.340000
3,Routing_2,M2,2.0,0.333333,0.077273,0.666667,0.154545
4,Routing_2,M3,3.0,0.333333,0.077273,1.000000,0.231818
5,Routing_2,M1,5.0,0.333333,0.077273,1.666667,0.386364
6,Routing_3,M3,4.0,0.300000,0.069545,1.200000,0.278182
7,Routing_3,M1,2.0,0.300000,0.069545,0.600000,0.139091
8,Routing_3,M2,3.0,0.300000,0.069545,0.900000,0.208636


In [11]:
# Ergänzung: Mittlere Bearbeitungszeit je Maschine
df_machine = (
    df_merged.groupby("Machine")["λj * d_jm"].sum() / lambda_gesamt
).reset_index(name="Mittlere Bearbeitungszeit")

# Ausgabe
print("\nMittlere Bearbeitungszeit je Maschine (gewichtet mit λj):")
df_machine 


Mittlere Bearbeitungszeit je Maschine (gewichtet mit λj):


,Machine,Mittlere Bearbeitungszeit
0,M1,3.000000
1,M2,3.400000
2,M3,3.666667


In [12]:
# Ergänzung: Mittlere Auslastung anhängen
df_machine["Mittlere Auslastung"] = df_machine["Mittlere Bearbeitungszeit"] * lambda_gesamt
df_machine

,Machine,Mittlere Bearbeitungszeit,Mittlere Auslastung
0,M1,3.000000,0.695455
1,M2,3.400000,0.788182
2,M3,3.666667,0.850000


In [13]:
# Berechnung der mittleren Verweilzeit je Maschine
df_machine["Mittlere Verweilzeit"] = (
    df_machine["Mittlere Bearbeitungszeit"] / (1 - df_machine["Mittlere Auslastung"])
)
df_machine

,Machine,Mittlere Bearbeitungszeit,Mittlere Auslastung,Mittlere Verweilzeit
0,M1,3.000000,0.695455,9.850746
1,M2,3.400000,0.788182,16.051502
2,M3,3.666667,0.850000,24.444444


In [14]:
# Berechnung der Service Rate je Maschine
df_machine["Service Rate"] = 1 / df_machine["Mittlere Bearbeitungszeit"]
df_machine


,Machine,Mittlere Bearbeitungszeit,Mittlere Auslastung,Mittlere Verweilzeit,Service Rate
0,M1,3.000000,0.695455,9.850746,0.333333
1,M2,3.400000,0.788182,16.051502,0.294118
2,M3,3.666667,0.850000,24.444444,0.272727


In [15]:
# Berechnung der Varianz der Verweilzeit pro Maschine
df_machine["Varianz"] = 1 / (df_machine["Service Rate"] - lambda_gesamt) ** 2
df_machine

,Machine,Mittlere Bearbeitungszeit,Mittlere Auslastung,Mittlere Verweilzeit,Service Rate,Varianz
0,M1,3.000000,0.695455,9.850746,0.333333,97.037202
1,M2,3.400000,0.788182,16.051502,0.294118,257.650721
2,M3,3.666667,0.850000,24.444444,0.272727,597.530865


In [16]:
varianz_durchlaufzeit = df_machine.Varianz.sum()
varianz_durchlaufzeit

np.float64(952.2187878026666)

In [17]:
standardabweichung_durchlaufzeit = math.sqrt(varianz_durchlaufzeit)
standardabweichung_durchlaufzeit 

30.858042514110753